## Import libraries

In [1]:
import gc
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

## Prepare data for model training

In [2]:
with open("../input/tps-may-data-preprocess-v3/TPS_May_Dataset_w_Org.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

del processed_data
gc.collect()

20

In [3]:
cat_cols = []
for col in tqdm(train_df.iloc[:,0:50].columns):
    train_ = train_df[col].unique().tolist()
    test_ = test_df[col].unique().tolist()
    
    train_.sort()
    test_.sort()
    
    if train_ == test_:
        cat_cols.append(col)

print(len(cat_cols))
cat_cols

100%|██████████| 50/50 [00:00<00:00, 1164.26it/s]

37


['feature_0',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_9',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18',
 'feature_20',
 'feature_21',
 'feature_22',
 'feature_23',
 'feature_25',
 'feature_26',
 'feature_27',
 'feature_28',
 'feature_29',
 'feature_32',
 'feature_33',
 'feature_34',
 'feature_35',
 'feature_36',
 'feature_38',
 'feature_39',
 'feature_42',
 'feature_43',
 'feature_44',
 'feature_46',
 'feature_49']

In [4]:
train_df[cat_cols] = train_df[cat_cols].astype(int)
test_df[cat_cols] = test_df[cat_cols].astype(int)

for col in tqdm(cat_cols):
    dummy_val = pd.get_dummies(train_df[col], prefix='col')
    train_df = pd.concat([train_df, dummy_val], axis=1)
    train_df.drop([col], inplace=True, axis=1)

print("train_df: {}".format(train_df.shape))

for col in tqdm(cat_cols):
    dummy_val = pd.get_dummies(test_df[col], prefix='col')
    test_df = pd.concat([test_df, dummy_val], axis=1)
    test_df.drop([col], inplace=True, axis=1)

print("test_df: {}".format(test_df.shape))

  3%|▎         | 1/37 [00:00<00:05,  6.33it/s]

train_df: (99918, 1886)


100%|██████████| 37/37 [00:06<00:00,  5.52it/s]

test_df: (50000, 1885)


In [5]:
Xtrain = train_df.loc[:, train_df.columns != 'target'].values
Ytrain = train_df['target'].values
Ytrain_oh = pd.get_dummies(train_df['target']).values
Xtest = test_df.values

print("Xtrain: {} \nYtrain: {} \nYtrain_oh: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, 
                                                                   Ytrain_oh.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

Xtrain: (99918, 1885) 
Ytrain: (99918,) 
Ytrain_oh: (99918, 4) 
Xtest: (50000, 1885)


100

## Build and validate the model

In [6]:
FOLD = 10
NUM_SEED = 3

# Prediction Clipping Thresholds
p_min = 0.025
p_max = 1 - p_min

np.random.seed(2021)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_xgb = np.zeros((Ytrain.shape[0], 4))
y_pred_final_xgb = np.zeros((Xtest.shape[0], 4))
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain, Ytrain)):
        counter += 1

        train_x, train_y, train_y_oh = Xtrain[train], Ytrain[train], Ytrain_oh[train]
        val_x, val_y, val_y_oh = Xtrain[val], Ytrain[val], Ytrain_oh[val]
        
        model = XGBClassifier(
            objective='multi:softmax',
            eval_metric='mlogloss',
            booster='gbtree',
            sample_type='weighted',
            tree_method='gpu_hist',
            grow_policy='lossguide',
            use_label_encoder=False,
            num_round=5000,
            num_class=4,
            #max_depth=10, 
            max_leaves=53,
            learning_rate=0.0982,
            subsample=0.76,
            colsample_bytree=0.52,
            min_child_weight=10,
            reg_lambda=0.001515,
            verbosity=0
        )
        
        model.fit(train_x, train_y, eval_set=[(train_x, train_y), (val_x, val_y)], 
                  early_stopping_rounds=200, verbose=50)

        y_pred = model.predict_proba(val_x, iteration_range=(0, model.best_iteration))
        y_pred = np.clip(y_pred, p_min, p_max)
        y_pred_meta_xgb[val] += y_pred
        y_pred_final_xgb += model.predict_proba(Xtest, iteration_range=(0, model.best_iteration))
        
        score = log_loss(val_y_oh, y_pred)
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))

y_pred_meta_xgb = y_pred_meta_xgb / float(NUM_SEED)
y_pred_final_xgb = y_pred_final_xgb / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

[0]	validation_0-mlogloss:1.34610	validation_1-mlogloss:1.34713
[50]	validation_0-mlogloss:1.05478	validation_1-mlogloss:1.09714
[99]	validation_0-mlogloss:1.02020	validation_1-mlogloss:1.09469


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


Seed-85 | Fold-0 | OOF Score: 1.0945829360442365
[0]	validation_0-mlogloss:1.34604	validation_1-mlogloss:1.34690
[50]	validation_0-mlogloss:1.05521	validation_1-mlogloss:1.09602
[99]	validation_0-mlogloss:1.02177	validation_1-mlogloss:1.09367


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


Seed-85 | Fold-1 | OOF Score: 1.0935679915531862
[0]	validation_0-mlogloss:1.34591	validation_1-mlogloss:1.34699
[50]	validation_0-mlogloss:1.05454	validation_1-mlogloss:1.09676
[99]	validation_0-mlogloss:1.02059	validation_1-mlogloss:1.09423


...

Seed-0 | Fold-7 | OOF Score: 1.095336206188743
[0]	validation_0-mlogloss:1.34600	validation_1-mlogloss:1.34691
[50]	validation_0-mlogloss:1.05409	validation_1-mlogloss:1.09563
[99]	validation_0-mlogloss:1.01965	validation_1-mlogloss:1.09355


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


Seed-0 | Fold-8 | OOF Score: 1.0936177039501986
[0]	validation_0-mlogloss:1.34596	validation_1-mlogloss:1.34702
[50]	validation_0-mlogloss:1.05472	validation_1-mlogloss:1.09943
[99]	validation_0-mlogloss:1.01979	validation_1-mlogloss:1.09675


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


Seed-0 | Fold-9 | OOF Score: 1.0967506439570562

Seed: 0 | Aggregate OOF Score: 1.0947983362090192


Aggregate OOF Score: 1.0950519012822286


In [7]:
del model
gc.collect()

769

In [8]:
np.savez_compressed('./XGB_Meta_Features.npz',
                    y_pred_meta_xgb=y_pred_meta_xgb, 
                    oof_score=oof_score,
                    y_pred_final_xgb=y_pred_final_xgb)

## Create submission file

In [9]:
y_pred_final_xgb = np.clip(y_pred_final_xgb, p_min, p_max)

test_df = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")
submit_df = pd.DataFrame()
submit_df['id'] = test_df['id']
submit_df['Class_1'] = y_pred_final_xgb[:,0]
submit_df['Class_2'] = y_pred_final_xgb[:,1]
submit_df['Class_3'] = y_pred_final_xgb[:,2]
submit_df['Class_4'] = y_pred_final_xgb[:,3]
submit_df.head()

,id,Class_1,Class_2,Class_3,Class_4
0,100000,0.084123,0.603899,0.188721,0.123256
1,100001,0.074170,0.661333,0.167542,0.096955
2,100002,0.085404,0.639757,0.179414,0.095425
3,100003,0.078688,0.477130,0.341408,0.102774
4,100004,0.074873,0.626725,0.203206,0.095197


In [10]:
submit_df.to_csv("./XGB_submission.csv", index=False)